CSE 158/258 - Recommender Sys & Web Mining - FA24
# Assignment 2 Report

Andrew Choi (A69033628), Chanbin Na (AA18087468), Jonghee Chun (A69033997), Kenny Hwang (A99021639)

[Github Repo](https://github.com/cjychoi/cse258-fa24/tree/main)

## 1. Dataset and Exploratory Analysis

**Dataset: Food.com Recipes and Interactions**\
Source: https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions

This dataset includes over 180K+ recipes and 700K+ recipe reviews spanning 18 years of user interactions and uploads on Food.com. For our analysis, we will utilize two files: RAW_interactions.csv and RAW_recipes.csv. We selected this dataset due to its rich and diverse information, such as ratings, reviews, tags, nutritional details, ingredients, steps, and descriptions.

<img src="img/dataset_img1.png" width="700">

Notably, the ratings appear to be skewed towards 5 out of 5, so we plan to treat ratings of 4 as indicative of a 'bad' review.


## 2. Predictive Task and Evaluation

## 3. Model Description

## 4. Related Literature

## 5. Results and Conclusions

## 6. References

**Food.com Recipes and Interactions (Kaggle)**\
Crawled data from Food.com (GeniusKitchen) online recipe aggregator\
https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=PP_recipes.csv


**Generating Personalized Recipes from Historical User Preferences**\
Bodhisattwa Prasad Majumder*, Shuyang Li*, Jianmo Ni, Julian McAuley
EMNLP, 2019\
https://www.aclweb.org/anthology/D19-1613/\
https://aclanthology.org/D19-1613.pdf
